# Datawhys Overall Attendance Tracker

This notebook tracks interns' session sign-ins over multiple days.

## Setup

<div class="alert alert-block alert-info">
<b>Instructor:</b> Comment out weeks or days which haven't happened yet to get a better summary view.
</div>

In [26]:
ALL_SESSION_DATES = [ 
  '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08',
  '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', 
]

<div class="alert alert-block alert-info">
<b>Instructor:</b> Set the times for the session periods to check.
</div>

In [27]:
SESSION_PERIODS = [
  ('10:00:00', '12:00:00'),
  ('13:00:00', '14:20:00'),
  ('14:40:00', '16:00:00'),
  # ('13:00:00', '16:00:00'),
]

<div class="alert alert-block alert-info">
<b>Instructor:</b> Run All Cells (from Run menu)
</div>

In [28]:
# FUNCTIONS

SIGN_IN_SHEET_URL = 'https://docs.google.com/spreadsheets/d/1bwZIJ2OSsIi9JY2a869AtJ2Oy5YYbbKWsQRPpfaOC10/edit#gid=427939934'

INTERNS = [
  ["Aa'Miyah Seals", 'aamiyah'],
  ['Affie Asante', 'affie'],
  ['Amier Cooper', 'amier'],
  ["Ashanti Stewart", 'ashanti'],
  ['Erielle Culp', 'erielle'],
  ['Maya McDonald', 'maya'],
  ['Nate Banda', 'nate'],
  ['Neville Chirewa', 'neville'],
  ['Prada House', 'prada'],
  ['Tai Chambers', 'tai'],
  ['Takia Coulter', 'takia'],
  ['Victor Enyogai', 'victor'],
]

import pandas as pd
from datetime import datetime
from datetime import timedelta
from functools import reduce

DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'
TIME_FORMAT = '%H%M'

def create_interns_df(interns=INTERNS):
  interns_df = pd.DataFrame(interns, columns=['Name', 'Username'])
  return interns_df.assign(lname = interns_df.Name.str.split(' ').str[-1]) \
                   .sort_values('lname') \
                   .drop('lname', axis=1)

INTERNS_DF = create_interns_df()

def create_sign_in_df(url=SIGN_IN_SHEET_URL):
  csv_url = url.replace('/edit#gid=', '/export?format=csv&gid=')
  sign_in_df = pd.read_csv(csv_url, parse_dates=['Timestamp']) \
                 .sort_values('Timestamp')
  return sign_in_df

SIGN_IN_DF = create_sign_in_df()

def get_session_sign_ins(date, start_time, end_time, sign_ins=SIGN_IN_DF, interns=INTERNS_DF):
  start_time = datetime.strptime('{} {}'.format(date, start_time), DATETIME_FORMAT)
  end_time = datetime.strptime('{} {}'.format(date, end_time), DATETIME_FORMAT)
  
  session_name = f"{start_time.strftime('%Y-%m-%d %H%M')}-{end_time.strftime(TIME_FORMAT)}"

  # 15 minutes before session start
  sign_in_start = start_time - timedelta(minutes=15)
  # end of session
  sign_in_end = end_time
  # 15 minutes after session start
  late_time = start_time + timedelta(minutes=15)

  # Filter only sign-ins between start and end
  session_sign_ins_df = sign_ins[(sign_in_start <= sign_ins.Timestamp) & (sign_ins.Timestamp <= sign_in_end)]
  # If multiple sign-ins, only keeping earliest
  session_sign_ins_df = session_sign_ins_df.sort_values('Timestamp') \
                                           .drop_duplicates(subset=['Your Name'])

  # Build df with NaT timestamp if no sign-in
  attendance_df = interns[['Name']]
  attendance_df = attendance_df.merge(session_sign_ins_df, how='left', left_on='Name', right_on='Your Name') \
                               .drop('Your Name', axis=1) \
                               .rename(columns = {'Timestamp': session_name}, inplace = False)
  return attendance_df

def overall_attendance(session_dates=ALL_SESSION_DATES, session_periods=SESSION_PERIODS):
  dfs = []
  for date in session_dates:
    for period in session_periods:
      dfs.append(get_session_sign_ins(date=date, start_time=period[0], end_time=period[1]))

  df_merged = reduce(lambda left, right: pd.merge(left, right, on=['Name']), dfs)
  return df_merged

## Attendance Summary

Track students with multiple missed check-ins.

In [29]:
attendance_df = overall_attendance()

# Overrides for present but forgot to check in
attendance_df.at[4,'2023-06-05 1000-1200'] = 'Present' #Amier
attendance_df.at[4,'2023-06-05 1300-1420'] = 'Present' #Amier
attendance_df.at[4,'2023-06-05 1440-1600'] = 'Present' #Amier
attendance_df.at[10,'2023-06-05 1000-1200'] = 'Present' #Aa'Miyah
attendance_df.at[10,'2023-06-05 1300-1420'] = 'Present' #Aa'Miyah
attendance_df.at[10,'2023-06-05 1440-1600'] = 'Present' #Aa'Miyah

In [30]:
attendance_df.insert(1, 'Missed Sessions', attendance_df.isnull().sum(axis=1))
attendance_df.style.applymap(lambda x: '' if x==x else 'background-color: rgb(255, 0, 0, 0.2);')

,Name,Missed Sessions,2023-06-05 1000-1200,2023-06-05 1300-1420,2023-06-05 1440-1600,2023-06-06 1000-1200,2023-06-06 1300-1420,2023-06-06 1440-1600,2023-06-07 1000-1200,2023-06-07 1300-1420,2023-06-07 1440-1600,2023-06-08 1000-1200,2023-06-08 1300-1420,2023-06-08 1440-1600,2023-06-12 1000-1200,2023-06-12 1300-1420,2023-06-12 1440-1600,2023-06-13 1000-1200,2023-06-13 1300-1420,2023-06-13 1440-1600,2023-06-14 1000-1200,2023-06-14 1300-1420,2023-06-14 1440-1600,2023-06-15 1000-1200,2023-06-15 1300-1420,2023-06-15 1440-1600
0,Affie Asante,20,2023-06-05 11:10:45,NaT,2023-06-05 14:44:10,2023-06-06 10:02:35,2023-06-06 13:02:56,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,Nate Banda,19,2023-06-05 11:10:37,2023-06-05 13:01:36,2023-06-05 14:42:19,2023-06-06 10:02:25,2023-06-06 13:03:13,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,Tai Chambers,23,2023-06-05 11:10:32,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,Neville Chirewa,19,2023-06-05 11:10:26,2023-06-05 13:07:09,2023-06-05 14:42:50,2023-06-06 10:02:27,2023-06-06 13:16:46,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,Amier Cooper,19,Present,Present,Present,2023-06-06 10:12:09,2023-06-06 13:02:41,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
5,Takia Coulter,19,2023-06-05 11:10:13,2023-06-05 13:06:09,2023-06-05 14:44:27,2023-06-06 10:03:53,2023-06-06 13:04:44,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
6,Erielle Culp,19,2023-06-05 11:10:22,2023-06-05 13:06:02,2023-06-05 14:41:43,2023-06-06 10:02:32,2023-06-06 13:02:49,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7,Victor Enyogai,19,2023-06-05 11:10:31,2023-06-05 13:06:39,2023-06-05 14:42:22,2023-06-06 10:02:39,2023-06-06 13:00:42,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
8,Prada House,20,NaT,2023-06-05 13:06:00,2023-06-05 14:42:30,2023-06-06 10:14:22,2023-06-06 13:02:52,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
9,Maya McDonald,20,2023-06-05 11:10:40,NaT,2023-06-05 14:42:57,2023-06-06 10:02:54,2023-06-06 13:03:20,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
